## Processing Scripts and Descriptions for Input Files

We start with the FAIR repository here: https://github.com/njleach/FAIR/tree/47c6eec031d2edcf09424394dbb86581a1b246ba noting the specific commit.  Processing scripts place all raw produced files into the `data/processing_scripts` folder, and then are postprocessed and placed into both the `data` main folder as well as the `python_replication_data` folder.

#### Emissions Data
- data/rcmip_sspxx_emissions_1750_to_2500.csv
- data/python_replication_data/sspxxx_emissions

**Note doing this work on June 6, 2023 yieled small changes for some gases for ssp585 from previous files.  The gases involved were `ch2cl, chcl3, methyl_bromine, methyl_chlorine, so2, nox, co, nnvoc, bc, nh3, oc`**

We used the `data_retrieval` scripts to obtain emissions data

In [ ]:
# this is a code snippet -- for full code see lfr_fairv2_outputs.ipynb
for ssp in ["ssp119", "ssp126", "ssp245", "ssp370", "ssp585"]:
    df = data_retrieval.RCMIP_to_FaIR_input_emms(ssp)
    filename = "rcmip_" + ssp + "_emissions_1750_to_2500_python.csv"
    df.to_csv("notebooks/lfr_outputs_data/" + filename) # copy into data/processing_scripts

We then postprocess these to linearly interpolate between future decadal years as indicated by the FAIR scripts using Julia.

In [6]:
using DataFrames, CSVFiles, Query, Interpolations

for ssp in ["ssp119", "ssp126", "ssp245", "ssp370", "ssp585"]
    df = load(joinpath(@__DIR__, "rcmip_$(ssp)_emissions_1750_to_2500_python.csv")) |> DataFrame
    titles = names(df)
    titles[1] = "year"
    rename!(df, titles)

    for col in names(df)[2:end]
        idxs = (!ismissing).(df[:,col])
        itp = LinearInterpolation(df[:, :year][idxs], df[:,col][idxs])
        df[:,col] .= itp[df.year]
    end

    df |> save(joinpath(@__DIR__, "..", "rcmip_$(ssp)_emissions_1750_to_2500.csv"))

    titles[1] = "Variable"
    rename!(df, titles)
    df |> save(joinpath(@__DIR__, "..", "python_replication_data", "$(ssp)_emissions.csv"))
end

Finally we add headers to each file in the main `data` file with the following metadata, these files are direct inputs to MimiFAIRv2.

```
# File Description: RCMIP SSP119 emissions scenario from 1750-2500.
# "Code Source: Extracted using default Python model version of FAIR2.0, available at https://github.com/njleach/
# FAIR/tree/47c6eec031d2edcf09424394dbb86581a1b246ba"
# "Paper Reference: Leach et al. 2021. ""FaIRv2.0.0: a generalized impulse response model for climate
# uncertainty and future scenario exploration,"" Geoscientific Model Development. https://doi.org/10.5194/gmd-14-3007-2021"
```

#### Default Gas Cycle Parameters
- data/python_replication_data/gas_cycle_parameters.csv
- data/default_gas_cycle_parameters.csv

Starting with the FAIR repository here: https://github.com/njleach/FAIR/tree/47c6eec031d2edcf09424394dbb86581a1b246ba and noting the specific commit, we use the following Python script and reformat (transform) usnig Excel.

In [ ]:
# this is a code snippet -- for full code see lfr_fairv2_outputs.ipynb
gas_parameters = get_gas_parameter_defaults()
gas_parameters.to_csv("notebooks/lfr_outputs_data/default_gas_cycle_parameters_python.csv")

We copy this file directly into the `python_replication_data` folder as `default_gas_parameters`.  

We then postprocess to make compatible inputs to MimiFAIRv2 including (1) transpose rows and columns (2) select only gases of interest (don't need ones with `|` in title) and (3) add gas group label to produce `default_gas_cycle_parameters.csv`

#### Default Thermal Parameters

In [ ]:
thermal_parameters = get_thermal_parameter_defaults()
thermal_parameters.to_csv("notebooks/lfr_outputs_data/default_thermal_parameters_python.csv")

We copy this file directly into the `python_replication_data` folder as `default_thermal_parameters`.  